In [104]:
import datetime
import math
import os
import sys

import numpy as np
import pandas as pd
sys.path.append("d:\\FundResearch\\")
import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_JJTG_DATA
from quant_utils.performance import Performance
import datetime


In [105]:
# 导入数据
df_idx = pd.read_excel(r'E:\深度研究\QDII投资价值分析\债券轮动测算\输入数据.xlsx',sheet_name= '收盘价')
df_yield = pd.read_excel(r'E:\深度研究\QDII投资价值分析\债券轮动测算\输入数据.xlsx',sheet_name= '利率')
df_usdcnh = pd.read_excel(r'E:\深度研究\QDII投资价值分析\债券轮动测算\输入数据.xlsx',sheet_name= '汇率')
df_idx = df_idx.dropna().loc[~((df_idx == 0).any(axis=1))]
df_yield = df_yield.dropna().loc[~((df_yield == 0).any(axis=1))]
df_usdcnh = df_usdcnh.dropna().loc[~((df_usdcnh == 0).any(axis=1))]
df_idx['日期'] = df_idx['日期'].apply(lambda s: s.strftime('%Y-%m-%d'))
df_yield['日期'] = df_yield['日期'].apply(lambda s: s.strftime('%Y-%m-%d'))
df_usdcnh['日期'] = df_usdcnh['日期'].apply(lambda s: s.strftime('%Y-%m-%d'))
df_yield




,日期,中债国债到期收益率:10年,美国:国债到期收益率:10年
0,2009-01-05,2.7300,2.49
1,2009-01-06,2.6909,2.51
2,2009-01-07,2.6711,2.52
3,2009-01-08,2.6716,2.47
4,2009-01-09,2.6722,2.43
...,...,...,...
3916,2023-09-12,2.6230,4.27
3917,2023-09-13,2.6150,4.25
3918,2023-09-14,2.6075,4.29
3919,2023-09-15,2.6400,4.33


In [106]:
trade_dt = '2000-1-1'

# 取周末或月末交易日
dates_query = f"""
select 
TRADE_DT,
IF_QUARTER_END
from 
md_tradingdaynew 
where 
SECU_MARKET = 83 and (IF_TRADING_DAY = 1) 
and TRADE_DT >= '{trade_dt}'
ORDER BY TRADE_DT
"""
dates_df = DB_CONN_JJTG_DATA.exec_query(dates_query)
dates_df.rename(columns={'TRADE_DT':'日期'}, inplace=True)
dates_df['日期'] = dates_df['日期'].apply(lambda s: s.strftime('%Y-%m-%d'))
dates_df

,日期,IF_QUARTER_END
0,2000-01-04,0
1,2000-01-05,0
2,2000-01-06,0
3,2000-01-07,0
4,2000-01-10,0
...,...,...
24306,2099-12-25,0
24307,2099-12-28,0
24308,2099-12-29,0
24309,2099-12-30,0


In [107]:
# 确定中债、美债、利差温度
df_yield['中美利差'] = df_yield['中债国债到期收益率:10年'] - df_yield['美国:国债到期收益率:10年']
df_yield['中债温度'] = df_yield['中债国债到期收益率:10年'].rolling(240*4).rank(pct=True, ascending=False)*100
df_yield['美债温度'] = df_yield['美国:国债到期收益率:10年'].rolling(240*4).rank(pct=True, ascending=False)*100
df_yield['利差温度'] = df_yield['中美利差'].rolling(240*4).rank(pct=True)*100
df_yield = pd.merge(df_yield, dates_df, on= '日期').dropna()
df_yield

,日期,中债国债到期收益率:10年,美国:国债到期收益率:10年,中美利差,中债温度,美债温度,利差温度,IF_QUARTER_END
959,2013-02-08,3.5859,1.99,1.5959,36.875000,75.937500,75.833333,0
960,2013-02-19,3.5931,2.03,1.5631,36.041667,71.770833,73.750000,0
961,2013-02-20,3.5632,2.02,1.5432,40.416667,72.604167,71.770833,0
962,2013-02-21,3.5623,1.99,1.5723,40.520833,75.885417,74.062500,0
963,2013-02-22,3.5708,1.97,1.6008,39.270833,77.708333,76.250000,0
...,...,...,...,...,...,...,...,...
3432,2023-09-12,2.6230,4.27,-1.6470,93.333333,0.833333,1.145833,0
3433,2023-09-13,2.6150,4.25,-1.6350,94.427083,1.354167,1.406250,0
3434,2023-09-14,2.6075,4.29,-1.6825,95.572917,0.572917,0.625000,0
3435,2023-09-15,2.6400,4.33,-1.6900,90.625000,0.312500,0.625000,0


In [108]:
Step = 5   # 定义权重调整步长

# 根据温度确定权重
df_yield['中债权重'] = df_yield.apply(
    lambda s: math.ceil(s['利差温度']/Step)*Step/100 if s['中债温度'] < 90 else math.ceil(s['利差温度']/Step)*Step/100*0.5,
axis=1
)
df_yield['美债权重'] = df_yield.apply(
    lambda s: math.floor((100-s['利差温度'])/Step)*Step/100 if s['美债温度'] < 90 else math.floor((100-s['利差温度'])/Step)*Step/100*0.5,
axis=1
)
df_yield['货币权重'] = 1-df_yield['中债权重']-df_yield['美债权重']
df_yield = df_yield.reset_index(drop=True)   #重新从0开始编索引





In [109]:
# 季度观察一次，重新分配权重
df_yield.loc[0, 'IF_QUARTER_END'] = 1

for i in range(len(df_yield)):
    if df_yield['IF_QUARTER_END'].loc[i] == 1:
        df_yield['中债权重'].loc[i] = df_yield['中债权重'].loc[i]
        df_yield['美债权重'].loc[i] = df_yield['美债权重'].loc[i]
        df_yield['货币权重'].loc[i] = df_yield['货币权重'].loc[i]
    else:
        df_yield['中债权重'].loc[i] = df_yield['中债权重'].loc[i-1]
        df_yield['美债权重'].loc[i] = df_yield['美债权重'].loc[i-1]
        df_yield['货币权重'].loc[i] = df_yield['货币权重'].loc[i-1]

df_yield

C:\Users\xyzq\AppData\Local\Temp\ipykernel_27212\134626016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yield['中债权重'].loc[i] = df_yield['中债权重'].loc[i]
C:\Users\xyzq\AppData\Local\Temp\ipykernel_27212\134626016.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yield['美债权重'].loc[i] = df_yield['美债权重'].loc[i]
C:\Users\xyzq\AppData\Local\Temp\ipykernel_27212\134626016.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yie

,日期,中债国债到期收益率:10年,美国:国债到期收益率:10年,中美利差,中债温度,美债温度,利差温度,IF_QUARTER_END,中债权重,美债权重,货币权重
0,2013-02-08,3.5859,1.99,1.5959,36.875000,75.937500,75.833333,1,0.800,0.20,-5.551115e-17
1,2013-02-19,3.5931,2.03,1.5631,36.041667,71.770833,73.750000,0,0.800,0.20,-5.551115e-17
2,2013-02-20,3.5632,2.02,1.5432,40.416667,72.604167,71.770833,0,0.800,0.20,-5.551115e-17
3,2013-02-21,3.5623,1.99,1.5723,40.520833,75.885417,74.062500,0,0.800,0.20,-5.551115e-17
4,2013-02-22,3.5708,1.97,1.6008,39.270833,77.708333,76.250000,0,0.800,0.20,-5.551115e-17
...,...,...,...,...,...,...,...,...,...,...,...
2473,2023-09-12,2.6230,4.27,-1.6470,93.333333,0.833333,1.145833,0,0.025,0.95,2.500000e-02
2474,2023-09-13,2.6150,4.25,-1.6350,94.427083,1.354167,1.406250,0,0.025,0.95,2.500000e-02
2475,2023-09-14,2.6075,4.29,-1.6825,95.572917,0.572917,0.625000,0,0.025,0.95,2.500000e-02
2476,2023-09-15,2.6400,4.33,-1.6900,90.625000,0.312500,0.625000,0,0.025,0.95,2.500000e-02


In [110]:
#汇率转换
df_idx = pd.merge(df_idx, df_usdcnh, on= '日期')
df_idx['IEF_cnh'] = df_idx['IEF']*df_idx['汇率']
df_idx

# 获取中债、美债、货币标的每期收益率
df_idx_merge = pd.merge(df_idx, df_yield[['日期', '中债权重', '美债权重', '货币权重']], on='日期')
df_idx_ret_merge = df_idx_merge[['日期', 'H11077.SH', 'IEF', 'H11025.CSI', 'IEF_cnh']].set_index('日期').pct_change().fillna(0).reset_index()
df_idx_ret_merge = pd.merge(df_idx_ret_merge, df_yield[['日期', '中债权重', '美债权重', '货币权重']], on='日期')
df_idx_ret_merge

# 计算轮动组合每期收益率
df_idx_ret_merge['组合'] = (
    df_idx_ret_merge['中债权重'].shift(1).fillna(0)*df_idx_ret_merge['H11077.SH']
    + df_idx_ret_merge['美债权重'].shift(1).fillna(0)*df_idx_ret_merge['IEF_cnh']
    + df_idx_ret_merge['货币权重'].shift(1).fillna(0)*df_idx_ret_merge['H11025.CSI']
)
df_idx_ret_merge



,日期,H11077.SH,IEF,H11025.CSI,IEF_cnh,中债权重,美债权重,货币权重,组合
0,2013-02-08,0.000000,0.000000,0.000000,0.000000,0.800,0.20,-5.551115e-17,0.000000
1,2013-02-19,0.001675,-0.003858,0.001015,-0.001972,0.800,0.20,-5.551115e-17,0.000946
2,2013-02-20,0.000427,0.002172,0.000107,0.001097,0.800,0.20,-5.551115e-17,0.000561
3,2013-02-21,0.001054,0.001979,0.000104,0.002445,0.800,0.20,-5.551115e-17,0.001332
4,2013-02-22,-0.000817,0.001223,0.000098,0.000356,0.800,0.20,-5.551115e-17,-0.000582
...,...,...,...,...,...,...,...,...,...
2473,2023-09-12,0.000529,0.001393,0.000048,0.000417,0.025,0.95,2.500000e-02,0.000411
2474,2023-09-13,0.000969,0.001177,0.000049,0.000544,0.025,0.95,2.500000e-02,0.000543
2475,2023-09-14,0.001019,-0.002244,0.000050,-0.003450,0.025,0.95,2.500000e-02,-0.003251
2476,2023-09-15,-0.000961,-0.003106,0.000051,-0.003243,0.025,0.95,2.500000e-02,-0.003103


In [111]:
# 计算中债、美债、货币、轮动组合净值曲线
df_idx_merge = pd.DataFrame(df_idx_ret_merge['日期'])
df_idx_merge['中债'] = (df_idx_ret_merge['H11077.SH']+1).cumprod()
df_idx_merge['美债'] = (df_idx_ret_merge['IEF_cnh']+1).cumprod()
df_idx_merge['货币'] = (df_idx_ret_merge['H11025.CSI']+1).cumprod()
df_idx_merge['组合'] = (df_idx_ret_merge['组合']+1).cumprod()

df_idx_merge = df_idx_merge.merge(df_yield[['日期', '中债权重', '美债权重', '货币权重']]).merge(df_usdcnh)
df_idx_merge.set_index('日期', inplace=True)

df_idx_merge

,中债,美债,货币,组合,中债权重,美债权重,货币权重,汇率
日期,,,,,,,,
2013-02-08,1.000000,1.000000,1.000000,1.000000,0.800,0.20,-5.551115e-17,6.2325
2013-02-19,1.001675,0.998028,1.001015,1.000946,0.800,0.20,-5.551115e-17,6.2443
2013-02-20,1.002103,0.999123,1.001122,1.001507,0.800,0.20,-5.551115e-17,6.2376
2013-02-21,1.003159,1.001566,1.001227,1.002841,0.800,0.20,-5.551115e-17,6.2405
2013-02-22,1.002340,1.001923,1.001325,1.002258,0.800,0.20,-5.551115e-17,6.2351
...,...,...,...,...,...,...,...,...
2023-09-12,1.511333,1.271726,1.373789,1.688067,0.025,0.95,2.500000e-02,7.2835
2023-09-13,1.512798,1.272419,1.373856,1.688983,0.025,0.95,2.500000e-02,7.2789
2023-09-14,1.514340,1.268029,1.373925,1.683493,0.025,0.95,2.500000e-02,7.2701


In [112]:
# 计算风险收益指标
port_perf_zz = Performance(df_idx_merge['中债']).stats().T
port_perf_zz['名称'] = '中债'

port_perf_mz = Performance(df_idx_merge['美债']).stats().T
port_perf_mz['名称'] = '美债'

port_perf_hb = Performance(df_idx_merge['货币']).stats().T
port_perf_hb['名称'] = '货币'

port_perf_portfolio = Performance(df_idx_merge['组合']).stats().T
port_perf_portfolio['名称'] = '组合'

port_perf = pd.concat([port_perf_zz, port_perf_mz, port_perf_hb, port_perf_portfolio])
port_perf


,起始日期,结束日期,累计收益率,年化收益率,年化波动率,收益波动比,最大回撤,年化收益回撤比,最大回撤日,最大回撤修复,名称
0,2013-02-08,2023-09-18,0.510742,0.039744,0.024181,1.643646,0.068573,0.579598,20131120,302,中债
0,2013-02-08,2023-09-18,0.269125,0.022768,0.079982,0.284665,0.24674,0.092276,20220419,99999,美债
0,2013-02-08,2023-09-18,0.374198,0.030482,0.001735,17.567794,0.000342,89.100672,20131210,3,货币
0,2013-02-08,2023-09-18,0.684784,0.050509,0.057304,0.881418,0.08132,0.621114,20220419,104,组合


In [113]:
with pd.ExcelWriter(r'E:\深度研究\QDII投资价值分析\债券轮动测算\输出数据.xlsx') as a:
    df_idx_merge.to_excel(a, sheet_name= '净值')
    port_perf.to_excel(a, sheet_name= '绩效')
